## Analyse residuals

This notebook investigates the use of the Cochrane-Orcutt method to improve a linear model of  mutation against recombination by correcting for the auto-correlation of residuals.

In [ ]:
import numpy as np
import pandas as pd
import os
import datetime
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import adfuller
import statsmodels.tsa.api as smt
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages


projdir = "..."   # Path to shared directory
if not os.getcwd() == projdir:
    os.chdir(projdir)
from shared import recombination

path = "..."      # Path to data file
if not os.getcwd() == path:
    os.chdir(path)

Open datafile, clean data and calculate residuals from ordinary least squares linear regression.

The data files are produced by sample_ensembl_for_recombination.py or merge_male_and_female_recombination_rates.py.

In [ ]:
chrom = '1'
sex = 'sexav'       #options are male, female, sex-averaged
csv_filename = 'Recombination_data/recomb_table_all_sexes_ch' + chrom + '.csv'
data_table = pd.read_csv(csv_filename, sep=',', index_col=0)
data_table = recombination.correct_missing_data(data_table,'LOCF', sex)
variants_profiled = data_table.iloc[:, np.arange(5, 17)]
variant_counts = variants_profiled.sum(axis=1)
var_rates = variant_counts / 10000
std_col = 'stdrate_' + sex
std_rates = data_table[std_col].values
print('Avge. mutation rate ', np.mean(var_rates))
xvals = std_rates.reshape(-1, 1)
lmodel = LinearRegression()
lmodel.fit(xvals, var_rates)
residuals = var_rates - lmodel.predict(xvals)
print('Slope, intercept, mean of residuals = ',\
          '%.8f' % lmodel.coef_[0], '%.8f' % lmodel.intercept_, '%.12f' % np.mean(residuals))
residuals = residuals.values

Plot residuals

In [ ]:
with PdfPages("Article_references/correlation_plot_supp_a.pdf") as pdf:
    figa = plt.figure(figsize=(5,4))
    y = pd.Series(residuals)
    y.plot()
    figa.suptitle('Plot of residuals', fontsize=16)
    plt.xlabel('Base position')
    plt.ylabel('Residual value')
    d = pdf.infodict()
    d['Title'] = 'Plot of residuals ' + sex
    d['Author'] = 'H. Simon'
    d['Subject'] = 'Datafile: ' + csv_filename
    d['Keywords'] = 'Notebook: ' + 'Mutations and recombination - analyse residuals.ipynb'
    d['CreationDate'] = datetime.datetime.today()
    pdf.savefig(figa, orientation='landscape')
plt.show()

Plot autocorrelation

In [ ]:
with PdfPages("Article_references/correlation_plot_supp_b.pdf") as pdf:
    figb = plt.figure(figsize=(5,4))
    figb = smt.graphics.plot_acf(y, lags=50, alpha=None, title=None)
    figb.suptitle('Autocorrelation', fontsize=16)
    plt.xlabel('Lag')
    plt.ylabel('Covariance')
    d = pdf.infodict()
    d['Title'] = 'Plot of autocorrelation ' + sex
    d['Author'] = 'H. Simon'
    d['Subject'] = 'Datafile: ' + csv_filename
    d['Keywords'] = 'Notebook: ' + 'Mutations and recombination - analyse residuals.ipynb'
    d['CreationDate'] = datetime.datetime.today()
    pdf.savefig(figb, orientation='landscape')
plt.show()

Plot partial autocorrelation

In [ ]:
with PdfPages("Article_references/correlation_plot_supp_c.pdf") as pdf:
    figc = plt.figure(figsize=(5, 4))
    figc = smt.graphics.plot_pacf(y, lags=50, alpha=None, title=None)
    figc.suptitle('Partial autocorrelation', fontsize=16)
    plt.xlabel('Lag')
    plt.ylabel('Covariance')
    d = pdf.infodict()
    d['Title'] = 'Plot of partial autocorrelation ' + sex
    d['Author'] = 'H. Simon'
    d['Subject'] = 'Datafile: ' + csv_filename
    d['Keywords'] = 'Notebook: ' + 'Mutations and recombination - analyse residuals.ipynb'
    d['CreationDate'] = datetime.datetime.today()
    pdf.savefig(figc, orientation='landscape')
plt.show()

Test the residuals for stationarity using the Augmented Dickey-Fuller test, implemented as adfuller. Note that the null hypothesis (which we reject) is that the series of residuals is * not * stationary. Stationary data can be approximated with an ARMA model (Wold decomposition theorem).

In [ ]:
result = adfuller(residuals, regression='ct')
print('ADF Statistic: %f' % result[0])
print('p-value: %f'       % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f'    % (key, value))
print('AIC: %f'           % result[5])

Select the preferred AR model and determine AIC. This is for comparison purposes only. Code to evaluate ARMA models is in shared/recombination.py.

In [ ]:
selectd_order = smt.AR(residuals).select_order(maxlag=30, ic='aic', trend='nc')
print('Selected order for AR = ', selectd_order)
mdl2 = smt.ARMA(residuals, order=(selectd_order, 0)).fit(method='mle', trend='nc')
print('AIC for order ' + str(selectd_order) + ' = ', mdl2.aic)